# Module import and function defination

In [197]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import h,e
import transmonCPW

def update_mutual_capacitance(Maxwell_cap_matrix,conductor1_index, conductor2_index, new_cap):
    """
    Args:
        Maxwell_cap_matrix: Maxwell capacitance matrix from Q3D
        conductor1_index : index of conductor 1
        conductor2_index : index of conductor 2
        new_cap : new capacitance for replacement
    Return:
        Updated Maxwell capacitance matrix
    """
    
    if conductor1_index == conductor2_index:
        raise('Error. First and second conductor index must be different!')
    
    M = np.copy(Maxwell_cap_matrix)
    
    M[conductor1_index, conductor1_index] = M[conductor1_index, conductor1_index] + M[conductor1_index, conductor2_index] + new_cap
    M[conductor2_index, conductor2_index] = M[conductor2_index, conductor2_index] + M[conductor2_index, conductor1_index] + new_cap
    
    M[conductor1_index, conductor2_index] = -new_cap
    M[conductor2_index, conductor1_index] = -new_cap
        
    return M

def calculate_qubit_2bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitances.
    Arg:
        C_M : ndarray, Maxewll capacitance matrix
        index : dict,  index of each conductor in C_M, starting from 0
    Return:
        1D numpy array of C_bus1, C_bus2, C_shunt, C_sigam, beta_bus1, beta_bus2 and beta_read.
    """
    index_bus1 = index['bus1']
    index_bus2 = index['bus2']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    # find beta_bus1, beta_bus2 and beta_readout
    #, where beta_bus1 = C_bus1/C_sigma 
    # and C_sigma = C_bus1 + C_bus2 + C_read + C_shunt

    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_bus2, index_bus2] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    b2 = np.linalg.solve(C_M_1, -C_M[:, index_bus2])
    beta_bus2 = b2[index_island1] - b2[index_island2]
    
    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
      
    # find C_bus1, C_bus2, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1, beta_bus1]
                  ,[beta_bus2, beta_bus2-1, beta_bus2, beta_bus2]
                ,[beta_read, beta_read, beta_read-1, beta_read]  
                ,[1, 1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    
    out = np.append(c, np.array([C_sigma, beta_bus1, beta_bus2, beta_read]))
    
    return out

def calculate_qubit_1bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitances.
    Arg:
        C_M : ndarray, Maxewll capacitance matrix
        index : dict, index of each conductor in C_M, starting from 0
    Return:
        1D numpy array of C_bus1, C_shunt, C_sigam, beta_bus1, and beta_read.
    """
    index_bus1 = index['bus1']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
        
    # find C_bus1, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1]
                ,[beta_read,  beta_read-1, beta_read]  
                ,[1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
        
    out = np.append(c, np.array([C_sigma, beta_bus1, beta_read]))
    
    return out

def print_result_1bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_read = {c[1]:.3f} fF\n'
         + f'C_shunt = {c[2]:.3f} fF\n'
         + f'C_sigma = {c[3]:.3f} fF\n'
         + f'beta_bus1 = {c[4]:.3f}\n'
         + f'beta_read = {c[5]:.3f}')
    
def print_result_2bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_bus2 = {c[1]:.3f} fF\n' 
         + f'C_read = {c[2]:.3f} fF\n'
         + f'C_shunt = {c[3]:.3f} fF\n'
         + f'C_sigma = {c[4]:.3f} fF\n'
         + f'beta_bus1 = {c[5]:.3f}\n'
         + f'beta_bus2 = {c[6]:.3f}\n'
         + f'beta_read = {c[7]:.3f}')

def read_capacitance_matrix_csv_to_numpy(filepath, num_conductor, display_df=True):
    """
    Read Q3D capacitance matrix in cvs form and output it in numpy array. 
    Args:
        filepath : Str, full file path
        num_conductor: int, number of conductors
        display_df: boolean, disply dataframe or not
    """
    
    df = pd.read_csv(filepath, header=6, nrows=num_conductor, index_col=0, usecols=range(num_conductor+1))
    if display_df:
        display(df)
    C_M = df.to_numpy()
  
    return C_M

def get_Ec_GHz_from_cap_fF(cap):
    return e**2/2/cap/1e-15/h/1e9

# ZZ gate, qubit 

In [161]:
# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [74]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_QubitPadC_Q3D.csv'
CSFQ = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(CSFQ, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,158.77860,-97.66067,-2.39180,-57.64469,-0.25352
gnd,-97.66067,344.81040,-41.88761,-48.54764,-83.19365
island1,-2.39180,-41.88761,71.05150,-17.08321,-8.46331
island2,-57.64469,-48.54764,-17.08321,125.56610,-0.97516
read,-0.25352,-83.19365,-8.46331,-0.97516,93.34146


C_bus1 = -17.553 fF
C_read = 5.328 fF
C_shunt = 65.348 fF
C_sigma = 53.123 fF
beta_bus1 = -0.330
beta_read = 0.100
Ec = 0.36463278092306656 GHz


In [171]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_QubitPadT_Q3D.csv'
transmon = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(transmon, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,133.78190,-92.38573,-1.94789,-38.55561,-0.20832
gnd,-92.38573,331.49750,-43.49309,-47.30495,-74.34755
island1,-1.94789,-43.49309,71.62598,-16.54816,-8.40905
island2,-38.55561,-47.30495,-16.54816,104.64300,-0.94813
read,-0.20832,-74.34755,-8.40905,-0.94813,84.33580


C_bus1 = -13.634 fF
C_read = 4.809 fF
C_shunt = 59.262 fF
C_sigma = 50.438 fF
beta_bus1 = -0.270
beta_read = 0.095
Ec = 0.3840418179292488 GHz


In [253]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_bus350um.csv'
bus350um = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(bus350um, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,121.05030,-83.51384,-2.17964,-34.36814,-0.13098
gnd,-83.51384,348.45120,-63.86928,-56.00028,-72.96077
island1,-2.17964,-63.86928,103.10760,-26.38928,-8.46271
island2,-34.36814,-56.00028,-26.38928,119.45440,-0.72889
read,-0.13098,-72.96077,-8.46271,-0.72889,82.71109


C_bus1 = -14.335 fF
C_read = 4.309 fF
C_shunt = 78.467 fF
C_sigma = 68.442 fF
beta_bus1 = -0.209
beta_read = 0.063
Ec = 0.2830180642294425 GHz


In [254]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_largeG.csv'
mod7p5_largeG = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(mod7p5_largeG, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,183.33960,-128.05210,-5.47775,-47.76669,-0.28035
gnd,-128.05210,373.83790,-67.95404,-32.56430,-73.65653
island1,-5.47775,-67.95404,98.13900,-13.72449,-8.61106
island2,-47.76669,-32.56430,-13.72449,95.69699,-0.43286
read,-0.28035,-73.65653,-8.61106,-0.43286,83.41221


C_bus1 = -21.535 fF
C_read = 4.023 fF
C_shunt = 72.825 fF
C_sigma = 55.312 fF
beta_bus1 = -0.389
beta_read = 0.073
Ec = 0.3501976144436483 GHz


In [259]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_largeG_2.csv'
mod7p5_largeG_2 = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(mod7p5_largeG_2, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,242.69930,-150.97040,-22.03286,-66.37392,-0.68677
gnd,-150.97040,380.68730,-62.83069,-22.04527,-73.55152
island1,-22.03286,-62.83069,103.38800,-7.86103,-8.49926
island2,-66.37392,-22.04527,-7.86103,97.33856,-0.25754
read,-0.68677,-73.55152,-8.49926,-0.25754,83.42281


C_bus1 = -23.616 fF
C_read = 3.978 fF
C_shunt = 73.701 fF
C_sigma = 54.063 fF
beta_bus1 = -0.437
beta_read = 0.074
Ec = 0.35829191704551716 GHz


In [282]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_largeG_3.csv'
mod7p5_largeG_3 = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(mod7p5_largeG_3, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,259.58200,-149.61280,-23.03372,-83.67349,-0.68428
gnd,-149.61280,380.57990,-62.86164,-23.21674,-73.55020
island1,-23.03372,-62.86164,103.38380,-6.82302,-8.49842
island2,-83.67349,-23.21674,-6.82302,114.82350,-0.25886
read,-0.68428,-73.55020,-8.49842,-0.25886,83.41989


C_bus1 = -27.336 fF
C_read = 4.365 fF
C_shunt = 80.775 fF
C_sigma = 57.803 fF
beta_bus1 = -0.473
beta_read = 0.076
Ec = 0.3351053678525255 GHz


In [274]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_largeG_4.csv'
CM = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(CM, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,183.00200,-126.41060,-2.59293,-51.98851,-0.27621
gnd,-126.41060,372.35190,-48.05055,-52.99280,-73.27376
island1,-2.59293,-48.05055,75.29654,-15.18587,-7.84231
island2,-51.98851,-52.99280,-15.18587,123.17360,-0.99644
read,-0.27621,-73.27376,-7.84231,-0.99644,82.82472


C_bus1 = -16.925 fF
C_read = 4.682 fF
C_shunt = 66.045 fF
C_sigma = 53.801 fF
beta_bus1 = -0.315
beta_read = 0.087
Ec = 0.36003182982795295 GHz


In [285]:
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_largeG_5.csv'
CM = read_capacitance_matrix_csv_to_numpy(filepath, 5, display_df=True)

result = calculate_qubit_1bus_1read_equiv_cap(CM, conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')

,bus,gnd,island1,island2,read
bus,256.13620,-153.40870,-12.76857,-86.31126,-0.90851
gnd,-153.40870,374.72120,-47.83652,-28.10257,-73.80664
island1,-12.76857,-47.83652,75.51451,-5.42510,-7.87152
island2,-86.31126,-28.10257,-5.42510,121.20690,-0.36809
read,-0.90851,-73.80664,-7.87152,-0.36809,83.39549


C_bus1 = -24.593 fF
C_read = 4.764 fF
C_shunt = 68.913 fF
C_sigma = 49.085 fF
beta_bus1 = -0.501
beta_read = 0.097
Ec = 0.3946279939224378 GHz


In [301]:
CM = update_mutual_capacitance(CM, 0, 3, 100)
CM = update_mutual_capacitance(CM, 1, 3, 10)
CM = update_mutual_capacitance(CM, 2, 3, 16)
CM = update_mutual_capacitance(CM, 2, 0, 23)
result = calculate_qubit_1bus_1read_equiv_cap(CM,conductor_index)
print_result_1bus_1read(result)
print(f'Ec = {e**2/2/result[3]/1e-15/h/1e9} GHz')
CM

C_bus1 = -28.539 fF
C_read = 4.419 fF
C_shunt = 86.785 fF
C_sigma = 62.665 fF
beta_bus1 = -0.455
beta_read = 0.071
Ec = 0.30910735282748575 GHz


array([[ 280.05637, -153.4087 ,  -23.     , -100.     ,   -0.90851],
       [-153.4087 ,  356.61863,  -47.83652,  -10.     ,  -73.80664],
       [ -23.     ,  -47.83652,   96.32084,  -16.     ,   -7.87152],
       [-100.     ,  -10.     ,  -16.     ,  127.36797,   -0.36809],
       [  -0.90851,  -73.80664,   -7.87152,   -0.36809,   83.39549]])

# I101 CSFQ, CR paper

In [49]:
# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [50]:
filepath = r'Z:\User\Jaseung\projects\ZZgate\Q3D\ZZgate_I101_CSFQ.csv'
C_M_I101_CSFQ = read_capacitance_matrix_csv_to_numpy(filepath, 5)

cap_result_I101_CSFQ = calculate_qubit_1bus_1read_equiv_cap(C_M_I101_CSFQ, conductor_index)
print_result_1bus_1read(cap_result_I101_CSFQ)
# C_QB  = 6.5 fF, C_QR = 2.8 fF

,bus,gnd,island1,island2,read
bus,95.48912,-77.86762,-1.24333,-15.94358,-0.07352
gnd,-77.86762,372.94720,-65.75782,-62.77826,-64.56598
island1,-1.24333,-65.75782,101.74740,-26.47125,-6.56720
island2,-15.94358,-62.77826,-26.47125,107.50170,-0.68268
read,-0.07352,-64.56598,-6.56720,-0.68268,72.09665


C_bus1 = -7.034 fF
C_read = 3.076 fF
C_shunt = 69.453 fF
C_sigma = 65.495 fF
beta_bus1 = -0.107
beta_read = 0.047


In [13]:
import transmonCPW
from scipy.constants import h,e,k

In [119]:
Ec = get_Ec_GHz_from_cap_fF(result[3])
q = transmonCPW.Transmon.in_GHz(13, Ec)

In [120]:
Ec

0.300483302301303

In [121]:
q.f01/1e6

5270.691503812834

In [122]:
q.anharmonicity/1e6

-349.8107414289418

# Mod 3

In [123]:
Mod3 = np.array([[0.28888, -0.0434,-0.0157,-0.0719,-0.0158]
                 ,[-0.0434, 0.0924,-0.00797,-0.0386,-0.0577]
                 ,[-0.0157,-0.00797, 0.0242,-0.000101,-0.00179]            
                 ,[-0.07198,-0.0386,-0.000101,0.119,-0.00803]
                 ,[-0.0158,-0.05778,-0.00179,-0.00803,0.0242]
                ])
Mod3 = Mod3 * 1e3
conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' :3 
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result = calculate_qubit_1bus_1read_equiv_cap(Mod3, conductor_index)
print_result_1bus_1read(result)


C_bus1 = 31.349 fF
C_read = 4.734 fF
C_shunt = 34.748 fF
C_sigma = 70.832 fF
beta_bus1 = 0.443
beta_read = 0.067


In [94]:
Ec = get_Ec_GHz_from_cap_fF(result[3])
q = transmonCPW.Transmon.in_GHz(11.4, Ec)
q.anharmonicity/1e6

-319.5507373346004

# Mod 4, 1bus-1read

In [97]:
C_M_mod4 = np.array([[291.34,-51.173,-17.152,-58.758,-17.068]
               ,[-51.173,92.843,-8.6623,-30.712,-0.0641]
               ,[-17.152,-8.6623,25.913,-0.0779,-0.00136]
               ,[-58.758,-30.712,-0.07791,99.027,-8.6762]
               ,[-17.068,-0.0641,-0.001367,-8.6762, 25.913]
               ])

conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' :3 
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result = calculate_qubit_1bus_1read_equiv_cap(C_M_mod4, conductor_index)
print_result_1bus_1read(result)

C_bus1 = -4.099 fF
C_read = 4.499 fF
C_shunt = 62.850 fF
C_sigma = 63.250 fF
beta_bus1 = -0.065
beta_read = 0.071


In [104]:
Ec = get_Ec_GHz_from_cap_fF(result[3])
q = transmonCPW.Transmon.in_GHz(10.4, Ec)
q.f01/1e9

4.7190626880612845

In [105]:
q.anharmonicity/1e6

-366.1916325955258

# Mod 7, 1bus-1read

In [109]:
Mod7_oldppt = np.array([[568.7, -43.028,-118.33,-39.898,-117.51]
               ,[-43.028,87.197,-16.9,-28.218,-1.1171]
               ,[-118.33,-16.9, 133.82, -1.126,-0.110033]
               ,[-39.898, -28.218,-1.126, 87.699, -17.034]
               ,[-117.51,-1.1171, -0.11033, -17.034, 133.73]
               ])

conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' :3 
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result_1bus_1read = calculate_qubit_1bus_1read_equiv_cap(Mod7_oldppt, conductor_index)
print_result_1bus_1read(result_1bus_1read)
np.sum(result_1bus_1read)

# Predicted values in the ppt seem wrong. e.g., g, chi are huge.

C_bus1 = -7.920 fF
C_read = 7.925 fF
C_shunt = 57.827 fF
C_sigma = 57.832 fF
beta_bus1 = -0.137
beta_read = 0.137


115.66502631980579

# Mod 7.5, 2bus-1read

In [114]:
# From Jay's ppt, MQC0V11 Summary_ADC.ppt
C_M = np.array([[107.94,   -0.072728, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-0.072728, 107.72, -95.966, -0.72699, -9.5856, -0.041242]
               ,[-96.56,   -95.966, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -0.72699, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -9.5856,-55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219,-0.041242, -95.917,   -7.0491, -0.61872, 103.03]
              ])  # from Jay's ppt

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
result = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result)
get_Ec_GHz_from_cap_fF(result[4])

C_bus1 = -4.248 fF
C_bus2 = -4.181 fF
C_read = 3.400 fF
C_shunt = 64.464 fF
C_sigma = 59.435 fF
beta_bu1 = -0.071
beta_bus2 = -0.070
beta_read = 0.057


0.3259050254764452

In [45]:
# My own Mod7.5
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_1.csv'
Mod_7p5_1= read_capacitance_matrix_csv_to_numpy(filepath, 6, display_df=True)

result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(Mod_7p5_1, conductor_index)
print_result_2bus_1read(result_2bus_1read)

,bus1,bus2,gnd,island1,island2,read
bus1,84.12624,-0.09242,-71.19074,-0.87456,-11.50396,-0.05208
bus2,-0.09242,88.28012,-75.31159,-0.88000,-11.51359,-0.05269
gnd,-71.19074,-75.31159,412.95030,-63.82632,-57.68698,-73.02149
island1,-0.87456,-0.88000,-63.82632,103.15110,-26.88716,-8.47016
island2,-11.50396,-11.51359,-57.68698,-26.88716,110.44020,-0.75749
read,-0.05208,-0.05269,-73.02149,-8.47016,-0.75749,82.78218


C_bus1 = -5.032 fF
C_bus2 = -5.034 fF
C_read = 4.067 fF
C_shunt = 72.758 fF
C_sigma = 66.758 fF
beta_bu1 = -0.075
beta_bus2 = -0.075
beta_read = 0.061


In [125]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }

filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_2.csv' # lareger GND plane
Mod_7p5_2= read_capacitance_matrix_csv_to_numpy(filepath, 6)
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(Mod_7p5_2, conductor_index)
print_result_2bus_1read(result_2bus_1read)

,bus1,bus2,gnd,island1,island2,read
bus1,130.05110,-0.08772,-117.41530,-0.85238,-11.48498,-0.04923
bus2,-0.08772,130.04180,-117.40910,-0.85247,-11.48951,-0.04929
gnd,-117.41530,-117.40910,614.68360,-65.88101,-59.61173,-115.08970
island1,-0.85238,-0.85247,-65.88101,103.51120,-26.72682,-8.44574
island2,-11.48498,-11.48951,-59.61173,-26.72682,110.75670,-0.73710
read,-0.04923,-0.04929,-115.08970,-8.44574,-0.73710,124.52270


C_bus1 = -5.038 fF
C_bus2 = -5.040 fF
C_read = 4.061 fF
C_shunt = 72.866 fF
C_sigma = 66.849 fF
beta_bu1 = -0.075
beta_bus2 = -0.075
beta_read = 0.061


In [129]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_1_vac.csv' # lareger GND plane
Mod7p5_1_vac= read_capacitance_matrix_csv_to_numpy(filepath, 6)
result = calculate_qubit_2bus_1read_equiv_cap(Mod7p5_1_vac, conductor_index)
print_result_2bus_1read(result)

,bus1,bus2,gnd,island1,island2,read
bus1,84.72878,-0.09250,-71.74216,-0.87504,-11.55217,-0.05220
bus2,-0.09250,88.92006,-75.90389,-0.88032,-11.55874,-0.05278
gnd,-71.74216,-75.90389,414.49250,-63.90671,-57.74837,-73.63248
island1,-0.87504,-0.88032,-63.90671,103.27270,-26.92179,-8.50045
island2,-11.55217,-11.55874,-57.74837,-26.92179,110.60460,-0.75809
read,-0.05220,-0.05278,-73.63248,-8.50045,-0.75809,83.43167


C_bus1 = -5.054 fF
C_bus2 = -5.054 fF
C_read = 4.083 fF
C_shunt = 72.871 fF
C_sigma = 66.846 fF
beta_bu1 = -0.076
beta_bus2 = -0.076
beta_read = 0.061


In [130]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
filepath = r'Z:/User/Jaseung/projects/ZZgate/Q3D/ZZgate_Mod7p5_1_3D.csv' # lareger GND plane
Mod7p5_1_3D= read_capacitance_matrix_csv_to_numpy(filepath, 6)
result = calculate_qubit_2bus_1read_equiv_cap(Mod7p5_1_3D, conductor_index)
print_result_2bus_1read(result)

,bus1,bus2,gnd,island1,island2,read
bus1,78.37977,-0.08718,-66.37413,-0.83166,-10.94480,-0.04701
bus2,-0.08718,82.16091,-70.31457,-0.83693,-10.87366,-0.04858
gnd,-66.37413,-70.31457,378.64200,-63.08294,-56.15770,-69.24143
island1,-0.83166,-0.83693,-63.08294,99.49774,-25.90376,-8.01563
island2,-10.94480,-10.87366,-56.15770,-25.90376,106.35550,-0.72377
read,-0.04701,-0.04858,-69.24143,-8.01563,-0.72377,78.07643


C_bus1 = -4.794 fF
C_bus2 = -4.758 fF
C_read = 3.840 fF
C_shunt = 70.051 fF
C_sigma = 64.339 fF
beta_bu1 = -0.075
beta_bus2 = -0.074
beta_read = 0.060


In [133]:
Mod7p5_1_730um = np.array([[84.15124,-0.08934,-71.46975,-0.87062,-11.49056,-0.05460]
                           ,[-0.08934,88.30017,-75.60000,-0.87623,-11.49921,-0.05525]
                           ,[-71.46975,-75.60000,417.90610,-63.69661,-57.57972,-73.44236]
                           ,[-0.87062,-0.87623,-63.69661,103.29970,-26.62650,-8.45201]
                           ,[-11.49056,-11.49921,-57.57972,-26.62650,110.56090,-0.73732]
                           ,[-0.05460,-0.05525,-73.44236,-8.45201,-0.73732,82.80042]
                          ])
result = calculate_qubit_2bus_1read_equiv_cap(Mod7p5_1_730um, conductor_index)
print_result_2bus_1read(result)

C_bus1 = -5.031 fF
C_bus2 = -5.032 fF
C_read = 4.065 fF
C_shunt = 72.694 fF
C_sigma = 66.696 fF
beta_bu1 = -0.075
beta_bus2 = -0.075
beta_read = 0.061


In [159]:
a=C_M[3,:]
a[:]
np.sum(a)

0.01563000000000425

In [156]:
np.sum(a)

1.3274400000000068